In [1]:
pip install trtokenizer

In [2]:
import pandas as pd
import re
import string
from trtokenizer.tr_tokenizer import SentenceTokenizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
from snowballstemmer import stemmer
from gensim.models import FastText
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import spacy
from spacy.lang.tr.stop_words import STOP_WORDS



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
df = pd.read_excel('tweetData.xlsx')

In [4]:
df.shape

(15604, 2)

In [5]:
df = df.dropna()

In [6]:
df.shape

(1976, 2)

In [7]:
def remove_special_characters(text):
    try:
      text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
      return text
    except:
        pass

def to_lowercase(text):
    try:
      return text.lower()
    except:
        pass

def remove_extra_spaces(text):
    try:
      return  ' '.join(text.split())
    except:
        pass

def remove_html_tags(text):
    try:
      clean = re.compile('<.*?>')
      return re.sub(clean, '', text)
    except:
        pass

def remove_urls(text):
    try:
      return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    except:
        pass

def remove_numbers(text):
    try:
      return re.sub(r'\d+', '', text)
    except:
        pass

def remove_punctuation(text):
    try:
      translator = str.maketrans('', '', string.punctuation)
      return text.translate(translator)
    except:
      pass

In [8]:
df['Tweets'] = df['Tweets'].apply(to_lowercase).apply(remove_extra_spaces).apply(remove_punctuation).apply(remove_html_tags).apply(remove_urls).apply(remove_numbers)

<ipython-input-8-acdcc727262d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweets'] = df['Tweets'].apply(to_lowercase).apply(remove_extra_spaces).apply(remove_punctuation).apply(remove_html_tags).apply(remove_urls).apply(remove_numbers)


In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.Tweets.loc[15595:15599]

15595    kimmiş lan rahatsız olacak söylesin infaz edel...
15596                              i̇nstasina infaz edelim
15597    üstad latin abd ülkelerinde denediler direk ka...
15598    bu iki oevladının açık adreslerini verin infaz...
15599     kıbrısın tamamı alınmalı   ada diye bize yutt...
Name: Tweets, dtype: object

In [10]:
df.head()

,Tweets,Etiket
0,aynı bayrak aynı göğün altındayız biriz berabe...,1.0
1,çok istekli olan bu erkekler lütfen erkek çoc...,1.0
2,kurtlerle komşumuz alrabayiz beraberiz siz yal...,2.0
3,müslümanın seni beni ırkı olmaz dini olur kims...,1.0
4,soramazsına hoşgeldiniz bu gün bilge ile berab...,2.0


In [21]:
# Cümle ve kelime ayrıştırıcı işlevler
def trtokenizer_sentence(text):
    try:
        return sent_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Sentence tokenization error: {e}")
        return []

def trtokenizer_word(text):
    try:
        return word_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Word tokenization error: {e}")
        return []

def remove_stopwords_spacy(text):

    turkish_stopwords = STOP_WORDS
    cleaned = [word for word in text if word not in turkish_stopwords]
    return cleaned

def snowballstemmer(word): # Changed function to receive a single word
  # Snowball Stemmer oluşturulması
  turkish_stemmer = stemmer("turkish")
  # Stem yapılması
  return turkish_stemmer.stemWord(word) # Return the stemmed word directly




# Cümle ve kelime ayrıştırma işlemlerini DataFrame'e uygulama
df['sentences'] = df['Tweets'].apply(trtokenizer_sentence)

# Kelimeleri çıkarmak için her cümleyi kelimelere böler ve tüm kelimeleri düz bir liste olarak toplar
df['words'] = df['sentences'].apply(lambda sentences: [word for sentence in sentences for word in trtokenizer_word(sentence)])

# Stop kelimeleri temizleme işlemini ve birleştirme işlemini DataFrame'e uygulama
df['cleaned_words'] = df['words'].apply(remove_stopwords_spacy)
df['lemmatized_words'] = df['cleaned_words'].apply(lambda words: [snowballstemmer(word) for word in words]) # Apply the stemming function to each word in the list
df['lemmatized_words'] = df['lemmatized_words'].apply(' '.join) # Convert the list of words back to a single string
# Sonuçları yazdırma
print(df.head())

                                              Tweets  Etiket  \
0  aynı bayrak aynı göğün altındayız biriz berabe...     1.0   
1  çok istekli olan bu erkekler  lütfen erkek çoc...     1.0   
2  kurtlerle komşumuz alrabayiz beraberiz siz yal...     2.0   
3  müslümanın seni beni ırkı olmaz dini olur kims...     1.0   
4  soramazsına hoşgeldiniz bu gün bilge ile berab...     2.0   

                                           sentences  \
0  [aynı bayrak aynı göğün altındayız biriz berab...   
1  [çok istekli olan bu erkekler  lütfen erkek ço...   
2  [kurtlerle komşumuz alrabayiz beraberiz siz ya...   
3  [müslümanın seni beni ırkı olmaz dini olur kim...   
4  [soramazsına hoşgeldiniz bu gün bilge ile bera...   

                                               words  \
0  [aynı, bayrak, aynı, göğün, altındayız, biriz,...   
1  [çok, istekli, olan, bu, erkekler, lütfen, erk...   
2  [kurtlerle, komşumuz, alrabayiz, beraberiz, si...   
3  [müslümanın, seni, beni, ırkı, olmaz, dini, ol...  

In [22]:

# Kelimeleri tokenize et
tokenize_metni = [metin.lower().split() for metin in df['lemmatized_words']]

# FastText modelini eğit
fasttext_model = FastText(vector_size=100, window=3, min_count=1, epochs=10)
fasttext_model.build_vocab(corpus_iterable=tokenize_metni)
fasttext_model.train(corpus_iterable=tokenize_metni, total_examples=len(tokenize_metni), epochs=10)

# Her metin için ortalama kelime vektörlerini hesapla
def ortalama_vektor(metin):
    kelimeler = metin.lower().split()
    vektörler = [fasttext_model.wv[kelime] for kelime in kelimeler if kelime in fasttext_model.wv]
    if vektörler:
        return np.mean(vektörler, axis=0)
    else:
        return np.zeros(100)

df['fasttext_vektor'] = df['lemmatized_words'].apply(ortalama_vektor)
print(df['fasttext_vektor'] )



0        [-0.46766388, -0.5355337, -0.27677226, 0.57788...
1        [-0.4987463, -0.5671375, -0.29400504, 0.614787...
2        [-0.29151997, -0.3338609, -0.17132072, 0.35930...
3        [-0.52346206, -0.5984965, -0.309755, 0.6468022...
4        [-0.36864412, -0.4214524, -0.21777983, 0.45556...
5        [-0.33981675, -0.388844, -0.20088246, 0.419318...
6        [-0.4625438, -0.52855724, -0.2734695, 0.570943...
7        [-0.52505404, -0.60040194, -0.3099583, 0.64734...
8        [-0.5199047, -0.5949341, -0.30917794, 0.641279...
9        [-0.55750597, -0.6373201, -0.32911843, 0.68809...
10       [-0.3763297, -0.43051916, -0.22263499, 0.46464...
11       [-0.40491945, -0.46395713, -0.24003538, 0.5004...
12       [-0.49123025, -0.56190884, -0.2909357, 0.60749...
13       [-0.3947949, -0.45144185, -0.2346867, 0.487807...
14       [-0.45993003, -0.5228559, -0.27189648, 0.56482...
15       [-0.34038648, -0.38929135, -0.20107602, 0.4198...
16       [-0.51671296, -0.591542, -0.30781028, 0.639611.